In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.utils import to_categorical


In [3]:
SIZE= 256
SEED_TRAINING=121
SEED_TESTING=197
SEED_VALIDATION=164
CHANNELS=3
n_classes=3
EPOCHS=20
BATCH_SIZE=16
input_shape=(SIZE, SIZE, CHANNELS)

In [4]:
train_datagen= ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        fill_mode='nearest')

In [5]:
validation_datagen= ImageDataGenerator(rescale=1./255)
test_datagen= ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        directory= '../input/potato-leaf-disease-dataset/Potato/Train',
        target_size= (255,256), # all images will be resized to 64*64
        batch_size= BATCH_SIZE,
        class_mode= 'categorical',
        color_mode="rgb")

In [7]:
validation_generator= validation_datagen.flow_from_directory(
            directory= '../input/potato-leaf-disease-dataset/Potato/Valid',
            target_size= (256,256),
            batch_size= BATCH_SIZE,
            class_mode= 'categorical',
            color_mode="rgb")

In [8]:
test_generator = test_datagen.flow_from_directory(
        directory='../input/potato-leaf-disease-dataset/Potato/Test',
        target_size = (256, 256),
        batch_size = BATCH_SIZE,
        class_mode = 'categorical',
        color_mode = "rgb"
)

BUILDING THE MODEL

In [9]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
model=keras.models.Sequential([
        keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation = 'relu', input_shape = input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.5),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Dropout(0.5),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding = 'same'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(32, activation ='relu'),
        keras.layers.Dense(n_classes, activation='softmax')
    ])

In [10]:
model.summary()

COMPILE THE MODEL

In [11]:
model.compile(
    optimizer='adam',
    loss= tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

TRAIN THE NETWORK

Accuracy for train_generator and validation_generator

In [12]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size,   #The 2 slashes division return rounded integer
        epochs = EPOCHS,
        validation_data = validation_generator, 
        validation_steps = validation_generator.n // validation_generator.batch_size 
        )

Accuracy for test_generator

In [ ]:
score= model.evaluate(test_generator)
print('Test loss:',score[0])
print('Test accuracy:',score[1])

History parameter


In [15]:
acc= history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss= history.history['val_loss']

In [16]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [17]:
train_generator.class_indices

In [18]:
class_names = list(train_generator.class_indices.keys())
class_names

In [19]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [20]:
model.save('final_model.h5')

In [21]:

def prediction(img):
    class_names = ['Early_blight', 'Healthy', 'Late_blight']

    my_image = image.load_img(img, SIZE)
    my_image = image.img_to_array(my_image)
    my_image = np.expand_dims(my_image, 0)

    out = np.round(model.predict(my_image)[0], 2)
    fig = plt.figure(figsize=(7, 4))
    plt.barh(class_names, out, color='lightgray',edgecolor='red', linewidth=1, height=0.5)

    for index, value in enumerate(out):
        plt.text(value/2 + 0.1, index, f"{100*value:.2f}%", fontweight='bold')

    plt.xticks([])
    plt.yticks([0, 1, 2], labels=class_names, fontweight='bold', fontsize=14)
    fig.savefig('pred_img.png', bbox_inches='tight')
    return plt.show()

In [22]:
img='../input/potato-leaf-disease-dataset/Potato/Test/Potato___healthy/Potato_healthy-26-_0_4635.jpg'
prediction(img)

In [23]:
img='../input/potato-leaf-disease-dataset/Potato/Test/Potato___Late_blight/00695906-210d-4a9d-822e-986a17384115___RS_LB 4026.JPG'
prediction(img)

In [24]:
img='../input/potato-leaf-disease-dataset/Potato/Test/Potato___Early_blight/08194ca3-f0b2-4aaa-8df8-5ec5ddc6696a___RS_Early.B 8151.JPG'
prediction(img)